# Library Imports

Importing necessary libraries and modules.

In [1]:
import numpy as np
import pandas as pd
import requests
import urllib.parse
import json

from datetime import datetime
from google.oauth2.service_account import Credentials

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# Data Scraping

Defining the variables that are required to upload the data to BigQuery tables.

In [2]:
target_table = "real_estate.jakarta"
target_table_2 = "real_estate.most_recent"
project_id = "jakarta-housing-price"
credential_file = "jakarta-housing-price-595a9cff2797.json"
credential = Credentials.from_service_account_file(credential_file)
job_location = "asia-southeast2"

In [3]:
query_most_recent = pd.read_gbq(f"SELECT * FROM `{project_id}.{target_table_2}`", project_id=project_id, credentials=credential)
query_most_recent["date"] = query_most_recent["date"].dt.tz_localize(None)
query_most_recent

,date,title,link,location,latitude,longitude,bedroom,bathroom,garage,land_m2,building_m2,price_idr,monthly_payment_idr,agent
0,2023-06-03,"Dijual Cepat Rumah Layar Permai Pik Uk 8x15, H...",https://www.rumah123.com/properti/jakarta-utar...,"Pantai Indah Kapuk, Jakarta Utara",-6.136197,106.90069,3,2,NaN,120,175,4.500000e+09,32000000.0,Budi Lim


Setting up the web scraping process using Selenium. It configures the Chrome driver with specific options, defines the website URL to scrape, and initializes lists to store the scraped data.

In [4]:
# Browser settings
website = "https://www.rumah123.com/jual/dki-jakarta/rumah/?sort=posted-desc&page=1#qid~a46c0629-67e4-410c-9c35-0c80e98987d9"
path = "chromedriver.exe"

service = Service(path)

options = Options()
options.add_argument("--headless")
options.add_argument("window-size=1920x1080")

driver = webdriver.Chrome(service=service, options=options)

# Lists to store the scraped data
title = []
link = []
location = []
price = []
bedrooms = []
bathrooms = []
garages = []
land_areas = []
building_areas = []
agent = []
date = []
scraped_timestamp = []

# Iterate through each page
for page in range(1, 101):
    print(f"Scraping page {page}")

    url = f"https://www.rumah123.com/jual/dki-jakarta/rumah/?sort=posted-desc&page={page}#qid~a46c0629-67e4-410c-9c35-0c80e98987d9"
    driver.get(url)

    # Pagination
    pagination = driver.find_element("class name", "ui-molecule-paginate")

    # Title
    title_elements = driver.find_elements("xpath", "//a[@title and h2]")
    titles = [element.text if element.text else float("nan") for element in title_elements]

    # Link
    link_elements = driver.find_elements("xpath", "//a[@title and h2]")
    links = [element.get_attribute("href") if element.get_attribute("href") else float("nan") for element in link_elements]

    # Location
    location_elements = driver.find_elements("xpath", "//a[@title and h2]/following-sibling::span")
    locations = [element.text if element.text else float("nan") for element in location_elements]

    # Price
    price_elements = driver.find_elements("class name", "card-featured__middle-section__price")
    prices = [element.text if element.text else float("nan") for element in price_elements]

    # Features
    features_elements = driver.find_elements("class name", "card-featured__middle-section__attribute")

    for index, element in enumerate(features_elements):
        print(f"House {index + 1} (Page {page}):")

        current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Scraped Timestamp: {current_timestamp}")
        scraped_timestamp.append(current_timestamp)

        print(f"Title: {titles[index] if index < len(titles) else 'N/A'}")
        title.append(titles[index] if index < len(titles) else 'N/A')

        print(f"Link: {links[index] if index < len(links) else 'N/A'}")
        link.append(links[index] if index < len(links) else 'N/A')

        print(f"Location: {locations[index] if index < len(locations) else 'N/A'}")
        location.append(locations[index] if index < len(locations) else 'N/A')

        print(f"Price: {prices[index] if index < len(prices) else 'N/A'}")
        price.append(prices[index] if index < len(prices) else 'N/A')
        
        # Bedroom
        try:
            bedroom_element = element.find_element("css selector", "div.ui-molecules-list__item:nth-child(1) span.attribute-text")
            bedroom = int(bedroom_element.text) if bedroom_element.text.isdigit() else 'N/A'
        except NoSuchElementException:
            bedroom = 'N/A'
        print(f"Bedroom: {bedroom}")
        bedrooms.append(bedroom)

        # Bathroom
        try:
            bathroom_element = element.find_element("css selector", "div.ui-molecules-list__item:nth-child(2) span.attribute-text")
            bathroom = int(bathroom_element.text) if bathroom_element.text.isdigit() else 'N/A'
        except NoSuchElementException:
            bathroom = 'N/A'
        print(f"Bathroom: {bathroom}")
        bathrooms.append(bathroom)

        # Garage
        try:
            garage_element = element.find_element("css selector", "div.ui-molecules-list__item:nth-child(3) span.attribute-text")
            garage = int(garage_element.text) if garage_element.text.isdigit() else 'N/A'
        except NoSuchElementException:
            garage = 'N/A'
        print(f"Garage: {garage}")
        garages.append(garage)

        # Land area
        try:
            land_area_element = element.find_element("xpath", ".//div[contains(text(), 'LT : ')]/span")
            land_area_text = land_area_element.text.strip()
            land_area = int(land_area_text.split()[0]) if land_area_text.split()[0].isdigit() else 'N/A'
        except NoSuchElementException:
            land_area = 'N/A'
        print(f"Land Area: {land_area}")
        land_areas.append(land_area)

        # Building area
        try:
            building_area_element = element.find_element("xpath", ".//div[contains(text(), 'LB : ')]/span")
            building_area_text = building_area_element.text.strip()
            building_area = int(building_area_text.split()[0]) if building_area_text.split()[0].isdigit() else 'N/A'
        except NoSuchElementException:
            building_area = 'N/A'
        print(f"Building Area: {building_area}")
        building_areas.append(building_area)

        # Agent & Date
        agent_date_elements = driver.find_elements("class name", "ui-organisms-card-r123-basic__bottom-section__agent")
        agents = [element.find_element("tag name", "p").text.strip() if element.find_element("tag name", "p").text.strip() else 'N/A' for element in agent_date_elements]
        dates = [element.find_elements("tag name", "p")[1].text.strip() if len(element.find_elements("tag name", "p")) > 1 and element.find_elements("tag name", "p")[1].text.strip() else 'N/A' for element in agent_date_elements]

        print(f"Agent: {agents[index] if index < len(agents) else 'N/A'}")
        agent.append(agents[index] if index < len(agents) else 'N/A')

        print(f"Date: {dates[index] if index < len(dates) else 'N/A'}")
        date.append(dates[index] if index < len(dates) else 'N/A')

        print("--------------------")

        # Check if conditions are met
        if titles[index] == query_most_recent["title"][0] and links[index] == query_most_recent["link"][0] and \
                locations[index] == query_most_recent["location"][0] and agents[index] == query_most_recent["agent"][0]:
            break

    # Break the outer loop if conditions are met
    else:
        continue
    break

driver.quit()

Scraping page 1
House 1 (Page 1):
Scraped Timestamp: 2023-06-03 13:14:11
Title: Rumah SHM 10x20, hdp selatan di Citra garden
Link: https://www.rumah123.com/properti/jakarta-barat/hos13908203/
Location: Citra Garden, Jakarta Barat
Price: Rp 2,5 Miliar
Cicilan : 17 Jutaan per bulan
Bedroom: 3
Bathroom: 2
Garage: 1
Land Area: 200
Building Area: 150
Agent: Sugihardi ARDI
Date: Diperbarui sejak 03 Juni, 2023
--------------------
House 2 (Page 1):
Scraped Timestamp: 2023-06-03 13:14:15
Title: RUMAH JELAMBAR TAMAN DUTA MAS KOMPLEK 2 LANTAI
Link: https://www.rumah123.com/properti/jakarta-barat/hos13908202/
Location: Jelambar, Jakarta Barat
Price: Rp 3,85 Miliar
Cicilan : 27 Jutaan per bulan
Bedroom: 4
Bathroom: 4
Garage: N/A
Land Area: 180
Building Area: 250
Agent: MELISA (LIPN)
Date: Diperbarui sejak 03 Juni, 2023
--------------------
House 3 (Page 1):
Scraped Timestamp: 2023-06-03 13:14:18
Title: RUMAH KLASIK MODERN SIAP HUNI GADING GRIYA LESTARI
Link: https://www.rumah123.com/properti/jakar

Creating a dictionary called `property_details` to store the scraped property details.

In [5]:
property_details = {
    "Title": title,
    "Link": link,
    "Location": location,
    "Bedroom": bedrooms,
    "Bathroom": bathrooms,
    "Garage": garages,
    "Land (m²)": land_areas,
    "Building (m²)": building_areas,
    "Price": price,
    "Agent": agent,
    "Date": date
}

for key, value in property_details.items():
    print("Key:", key)
    print("Length of Values:", len(value))
    print("---")

Key: Title
Length of Values: 30
---
Key: Link
Length of Values: 30
---
Key: Location
Length of Values: 30
---
Key: Bedroom
Length of Values: 30
---
Key: Bathroom
Length of Values: 30
---
Key: Garage
Length of Values: 30
---
Key: Land (m²)
Length of Values: 30
---
Key: Building (m²)
Length of Values: 30
---
Key: Price
Length of Values: 30
---
Key: Agent
Length of Values: 30
---
Key: Date
Length of Values: 30
---


# Data Cleaning

Defining a dictionary called `month_mapping` that maps Indonesian month names to their English equivalents.

In [6]:
month_mapping = {
    "(?i)Januari": "January",
    "(?i)Februari": "February",
    "(?i)Maret": "March",
    "(?i)April": "April",
    "(?i)Mei": "May",
    "(?i)Juni": "June",
    "(?i)Juli": "July",
    "(?i)Agustus": "August",
    "(?i)September": "September",
    "(?i)Oktober": "October",
    "(?i)November": "November",
    "(?i)Desember": "December"
}

Defining a function `convert_price` to convert the scraped price string into a numeric value.

In [7]:
def convert_price(price):
    number_split = price.split(" ")

    numeric = float(number_split[0].replace(",", "."))
    suffix = number_split[1]

    if "triliun" in suffix.lower():
        multiplier = 10**12
    elif "miliar" in suffix.lower():
        multiplier = 10**9
    elif "juta" in suffix.lower():
        multiplier = 10**6
    else:
        multiplier = 1

    numeric *= multiplier
    return numeric

Retrieving the latitude and longitude coordinates for specific locations in Jakarta using the Nominatim API and storing them in a dictionary. The dictionary is then saved as a JSON file.

In [8]:
jakarta_locations = ["Jakarta Pusat", "Jakarta Utara", "Jakarta Timur", "Jakarta Selatan", "Jakarta Barat"]

coordinates = {}

for loc in jakarta_locations:
    url = "https://nominatim.openstreetmap.org/search/" + urllib.parse.quote(loc) +"?format=json"
    response = requests.get(url).json()
    coordinates[loc] = {"latitude": response[0]["lat"], "longitude": response[0]["lon"]}

with open("jakarta_coordinates.json", "w") as file:
    file.write(json.dumps(coordinates))

Loading the JSON file containing the coordinates of Jakarta locations into a dictionary.

In [9]:
with open("jakarta_coordinates.json", "r") as file:
    coordinates = json.loads(file.read())

coordinates

{'Jakarta Pusat': {'latitude': '-6.18233995',
  'longitude': '106.84287153600738'},
 'Jakarta Utara': {'latitude': '-6.136197', 'longitude': '106.90069022428446'},
 'Jakarta Timur': {'latitude': '-6.26289085',
  'longitude': '106.88222894692834'},
 'Jakarta Selatan': {'latitude': '-6.28381815',
  'longitude': '106.80486325728526'},
 'Jakarta Barat': {'latitude': '-6.161569', 'longitude': '106.74389053715214'}}

Creating Pandas DataFrames from the scraped data. It performs various data cleaning and transformation steps such as replacing text, converting data types, applying the `convert_price` function, retrieving latitude and longitude coordinates from the `coordinates` dictionary, and reordering the columns.

In [10]:
df = pd.DataFrame({
    "Title": title,
    "Link": link,
    "Location": location,
    "Bedroom": bedrooms,
    "Bathroom": bathrooms,
    "Garage": garages,
    "Land m2": land_areas,
    "Building m2": building_areas,
    "Price": price,
    "Agent": agent,
    "Date": date,
    "Scraped Timestamp": scraped_timestamp
})

df["Date"] = df["Date"].str.replace("Diperbarui sejak ", "").str.replace(",", "")
df["Date"] = df["Date"].replace(month_mapping, regex=True)
df["Date"] = pd.to_datetime(df["Date"])

df["Price IDR"] = df["Price"].str.split("\n").str[0].str.replace("Rp ", "")
df["Price IDR"] = df["Price IDR"].apply(convert_price)

df["Monthly Payment IDR"] = df["Price"].str.split("\n").str[1].str.replace("Cicilan : ", "").str.replace(" per bulan", "")
df["Monthly Payment IDR"] = df["Monthly Payment IDR"].apply(convert_price)

df["Latitude"] = df["Location"].str.split(", ").str[-1].map(lambda x: coordinates.get(x, {}).get("latitude"))
df["Longitude"] = df["Location"].str.split(", ").str[-1].map(lambda x: coordinates.get(x, {}).get("longitude"))

df["Scraped Timestamp"] = pd.to_datetime(df["Scraped Timestamp"])

df = df.drop("Price", axis=1)
df = df.replace("N/A", np.nan)
df = df[["Date", "Title", "Link", "Location", "Latitude", "Longitude", "Bedroom", "Bathroom", "Garage", "Land m2", "Building m2", "Price IDR", "Monthly Payment IDR", "Agent", "Scraped Timestamp"]]
df.columns = df.columns.str.lower().str.replace(" ", "_")

for col in ["latitude", "longitude", "bedroom", "bathroom", "garage", "land_m2", "building_m2", "price_idr", "monthly_payment_idr"]:
    df[col] = df[col].astype(float)

condition = (
    (df["title"] == query_most_recent["title"][0]) &
    (df["link"] == query_most_recent["link"][0]) &
    (df["location"] == query_most_recent["location"][0]) &
    (df["agent"] == query_most_recent["agent"][0])
)
df = df[~condition]

df.head()

,date,title,link,location,latitude,longitude,bedroom,bathroom,garage,land_m2,building_m2,price_idr,monthly_payment_idr,agent,scraped_timestamp
0,2023-06-03,"Rumah SHM 10x20, hdp selatan di Citra garden",https://www.rumah123.com/properti/jakarta-bara...,"Citra Garden, Jakarta Barat",-6.161569,106.743891,3.0,2.0,1.0,200.0,150.0,2.500000e+09,17000000.0,Sugihardi ARDI,2023-06-03 13:14:11
1,2023-06-03,RUMAH JELAMBAR TAMAN DUTA MAS KOMPLEK 2 LANTAI,https://www.rumah123.com/properti/jakarta-bara...,"Jelambar, Jakarta Barat",-6.161569,106.743891,4.0,4.0,NaN,180.0,250.0,3.850000e+09,27000000.0,MELISA (LIPN),2023-06-03 13:14:15
2,2023-06-03,RUMAH KLASIK MODERN SIAP HUNI GADING GRIYA LES...,https://www.rumah123.com/properti/jakarta-utar...,"Cilincing, Jakarta Utara",-6.136197,106.900690,4.0,4.0,NaN,120.0,220.0,2.900000e+09,20000000.0,CITRA (HVOI),2023-06-03 13:14:18
3,2023-06-03,RUMAH KLASIK MODERN SIAP HUNI GADING GRIYA LES...,https://www.rumah123.com/properti/jakarta-utar...,"Cilincing, Jakarta Utara",-6.136197,106.900690,4.0,4.0,NaN,120.0,220.0,2.900000e+09,20000000.0,DEVIT (BDFT),2023-06-03 13:14:21
4,2023-06-03,RUMAH JELAMBAR BAGUS SIAP HUNI 2.5 LANTAI,https://www.rumah123.com/properti/jakarta-bara...,"Jelambar, Jakarta Barat",-6.161569,106.743891,3.0,3.0,NaN,60.0,100.0,1.600000e+09,11000000.0,SISCA (ZXPU),2023-06-03 13:14:25


In [11]:
most_recent = df[df["scraped_timestamp"] == df["scraped_timestamp"].min()]
most_recent

,date,title,link,location,latitude,longitude,bedroom,bathroom,garage,land_m2,building_m2,price_idr,monthly_payment_idr,agent,scraped_timestamp
0,2023-06-03,"Rumah SHM 10x20, hdp selatan di Citra garden",https://www.rumah123.com/properti/jakarta-bara...,"Citra Garden, Jakarta Barat",-6.161569,106.743891,3.0,2.0,1.0,200.0,150.0,2.500000e+09,17000000.0,Sugihardi ARDI,2023-06-03 13:14:11


# Data Storage

Uploading the DataFrames to the specified BigQuery tables.

In [12]:
schema = [
    {"name": "date", "type": "DATE"},
    {"name": "title", "type": "STRING"},
    {"name": "link", "type": "STRING"},
    {"name": "location", "type": "STRING"},
    {"name": "latitude", "type": "FLOAT64"},
    {"name": "longitude", "type": "FLOAT64"},
    {"name": "bedroom", "type": "FLOAT64"},
    {"name": "bathroom", "type": "FLOAT64"},
    {"name": "garage", "type": "FLOAT64"},
    {"name": "land_m2", "type": "FLOAT64"},
    {"name": "building_m2", "type": "FLOAT64"},
    {"name": "price_idr", "type": "FLOAT64"},
    {"name": "monthly_payment_idr", "type": "FLOAT64"},
    {"name": "agent", "type": "STRING"}
]

In [13]:
df_without_timestamp = df.copy()
df_without_timestamp = df_without_timestamp.drop("scraped_timestamp", axis=1)

df_without_timestamp.to_gbq(
    destination_table=target_table,
    project_id=project_id,
    if_exists="append",
    location=job_location,
    chunksize=10_000,
    progress_bar=True,
    credentials=credential,
    table_schema=schema
)

100%|██████████| 1/1 [00:00<?, ?it/s]


In [14]:
most_recent_without_timestamp = most_recent.copy()
most_recent_without_timestamp = most_recent_without_timestamp.drop("scraped_timestamp", axis=1)

most_recent_without_timestamp.to_gbq(
    destination_table=target_table_2,
    project_id=project_id,
    if_exists="replace",
    location=job_location,
    progress_bar=True,
    credentials=credential,
    table_schema=schema
)

100%|██████████| 1/1 [00:00<00:00, 994.15it/s]
